<a href="https://colab.research.google.com/github/heejvely/NLP_models/blob/main/Google_Sentence_Piece%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_vocab_%ED%8C%8C%EC%9D%BC_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 코드 참조 :https://github.com/paul-hyun/transformer-evolution/blob/master/tutorial/vocab_with_sentencepiece.ipynb

0. Pip install

필요한 패키지를 pip를 이용해 설치

In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 6.9 MB/s 


1. Google Drive Mount

colab에서는 컴퓨터 자원에 접근이 불가능하므로 Google Drive에 파일을 올려 놓은 후 로컬처럼 사용함.

In [4]:
# data 저장할 폴더
data_dir = '/content/drive/MyDrive/colab/NLP_모델/web-crawler/kowiki'

2. Imports

In [3]:
import os
import gzip
import pandas as pd
import sentencepiece as spm
import shutil

3. 폴더의 목록을 확인

In [5]:
os.listdir(data_dir)

['kowiki_20211228.csv']

4. CSV 파일을 TEXT 형태로 변환

Google SentencePiece에서 사용할 수 있도록 CSV를 TEXT 형태로 변환합니다.

In [9]:
# field larger than field limit: error로 인한 설정
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [11]:
in_file = '/content/drive/MyDrive/colab/NLP 모델/web-crawler/kowiki/kowiki_20211228.csv'
out_file = '/content/drive/MyDrive/colab/NLP 모델/kowiki.txt'
SEPERATOR = u'\u241D'
df = pd.read_csv(in_file, sep=SEPERATOR, engine='python')
with open(out_file,'w')  as f:
  for index, row in df.iterrows():
    f.write(row['text'])        # title과 text가 중복되므로 text만 저장
    f.write('\n\n\n\n')         # 구분자 - 위키 문서별로 구분하기 위함.

5. Vovab 만들기

#### 옵션
- input: 입력 corpus
- prefix: 저장할 모델 이름
- vocab_size:vocab 개수(기본 8,000에 스페셜 토큰 7개를 더해서 8,007개)
- max_sentence_length: 문장의 최대 길이
- pad_id, pad_piece: nknown token id, 값
- bos_id, bos_piece: begin of sentence token id, 값
- eos_id, eos_piece: end of sequence token id, 값
- user_defined_symbols: 사용자 정의 토큰

In [14]:
corpus = '/content/drive/MyDrive/colab/NLP_모델/kowiki.txt'
prefix = 'kowiki'
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [16]:
# model, vocab 저장
shutil.copy("kowiki.model", "/content/drive/MyDrive/colab/NLP_모델/kowiki_corpus/kowiki.model")
shutil.copy("kowiki.vocab", "/content/drive/MyDrive/colab/NLP_모델/kowiki_corpus/kowiki.vocab")

'/content/drive/MyDrive/colab/NLP_모델/kowiki_corpus/kowiki.vocab'

6 Vobab Test

생성된 vocab을 이용한 간단한 테스트 코드입니다.

In [17]:
vocab_file= '/content/drive/MyDrive/colab/NLP_모델/kowiki_corpus/kowiki.model'
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
         "겨울이 되어서 날씨가 무척 추워요.",
         "이번 성탄절은 화이트 크리스마스가 될까요?",
         "겨울에 감기 조심하시고 행복한 연말 되세요."
]

for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()

겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어', '서', '▁날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[3215, 3540, 637, 3554, 702, 3981, 3551, 107, 4151, 197, 3908, 3712, 3542]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성', '탄', '절', '은', '▁화', '이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[2915, 88, 3921, 3876, 3556, 268, 659, 1908, 3618, 759, 1432, 3750, 3712, 4215]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[3215, 3543, 200, 3558, 53, 3785, 3548, 3565, 3552, 241, 3826, 3555, 63, 3781, 453, 3636, 3712, 3542]



subword 단위로 잘 쪼개지는 것을 확인할 수 있음.